# 1 Multi-agent conversation

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-4o", "api_key":OPENAI_API_KEY, "temperature":1.0}

from autogen import ConversableAgent

## Conversation
Let's create our first agents and let them interact each other. 
The agents are Marcello and Luigi, and they support their favourite team, Roma and Lazio respectively.

In [2]:
marcello = ConversableAgent(
    name="marcello",
    system_message="""
    Your name is Marcello, and you are a funny AS ROMA football team supporter.
    You always use few words when interacting with other supporters.
    """,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

luigi = ConversableAgent(
    name="luigi",
    system_message=
    """
    Your name is Luigi and you are a funny SS LAZIO football team supporter.
    You always use few words when interacting with other supporters.
    """,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

Let's meet them!

In [3]:
# 2 interactions at maximum
chat_result = marcello.initiate_chat(
    recipient=luigi, 
    message="Hi Luigi, Roma won against Lazio last time, how do you feel?",
    max_turns=2,
)

marcello (to luigi):

Hi Luigi, Roma won against Lazio last time, how do you feel?

--------------------------------------------------------------------------------
luigi (to marcello):

Ciao Marcello! Sad, but... Forza Lazio sempre! 💙

--------------------------------------------------------------------------------
marcello (to luigi):

Ah, Luigi! Roma sempre avanti! 💛❤️

--------------------------------------------------------------------------------
luigi (to marcello):

Marcello, enjoy it... while it lasts. 😉 Forza Lazio! 💙

--------------------------------------------------------------------------------


We can extract some information from the previous interactions, like:

1. Chat history
2. Cost
3. Summary of the conversation

In [4]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': 'Hi Luigi, Roma won against Lazio last time, how do you feel?',
  'name': 'marcello',
  'role': 'assistant'},
 {'content': 'Ciao Marcello! Sad, but... Forza Lazio sempre! 💙',
  'name': 'luigi',
  'role': 'user'},
 {'content': 'Ah, Luigi! Roma sempre avanti! 💛❤️',
  'name': 'marcello',
  'role': 'assistant'},
 {'content': 'Marcello, enjoy it... while it lasts. 😉 Forza Lazio! 💙',
  'name': 'luigi',
  'role': 'user'}]


In [5]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-4o-2024-08-06': {'completion_tokens': 47,
                                                            'cost': 0.0011,
                                                            'prompt_tokens': 252,
                                                            'total_tokens': 299},
                                      'total_cost': 0.0011},
 'usage_including_cached_inference': {'gpt-4o-2024-08-06': {'completion_tokens': 47,
                                                            'cost': 0.0011,
                                                            'prompt_tokens': 252,
                                                            'total_tokens': 299},
                                      'total_cost': 0.0011}}


In [6]:
pprint.pprint(chat_result.summary) # default: last sent message

'Marcello, enjoy it... while it lasts. 😉 Forza Lazio! 💙'


## Get a better summary of the conversation

In [7]:
chat_result = marcello.initiate_chat(
    recipient=luigi, 
    message="Hi Luigi, Roma won against Lazio last time, how do you feel?", 
    max_turns=2, 
    summary_method="reflection_with_llm", # <---
    summary_prompt="Summarize the conversation", # <---
)

marcello (to luigi):

Hi Luigi, Roma won against Lazio last time, how do you feel?

--------------------------------------------------------------------------------
luigi (to marcello):

Ciao Marcello! Sad, but... Forza Lazio sempre! 💙

--------------------------------------------------------------------------------
marcello (to luigi):

Ah, Luigi! Roma sempre avanti! 💛❤️

--------------------------------------------------------------------------------
luigi (to marcello):

Marcello, enjoy it... while it lasts. 😉 Forza Lazio! 💙

--------------------------------------------------------------------------------


In [8]:
pprint.pprint(chat_result.summary)

("Marcello is happy about Roma's recent victory over Lazio and teases Luigi, "
 'who remains supportive of Lazio despite the loss.')


## Chat termination

Chat can be concluded with some termination conditions.

In [9]:
marcello = ConversableAgent(
    name="marcello_asroma",
    system_message=
    """Your name is Marcello, and you are a funny AS ROMA football team supporter.
    You always use few words when interacting with other supporters.
    You are walking down the street when at one point you pass a Lazio fan.
    The last game against Lazio was won by Roma 2-0.
    You don't have much time, so when you have to leave say "gotta go, goodbye!"
    """,
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "gotta go" in msg["content"].lower() or "goodbye" in msg["content"].lower()
)

luigi = ConversableAgent(
    name="luigi_sslazio",
    system_message=
    """Your name is Luigi and you are a funny SS LAZIO football team supporter.
    You always use few words when interacting with other supporters.
    You are walking down the street when at one point you pass a Lazio fan.
    The last game against Lazio was won by Roma 2-0.
    When you have to leave say "gotta go, goodbye!"
    """,
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "gotta go" in msg["content"].lower() or "goodbye" in msg["content"].lower()
)

In [10]:
chat_result = marcello.initiate_chat(
    recipient=luigi,
    message="Hi Luigi, Roma won against Lazio last time, how do you feel?", 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
    max_turns=6
)

marcello_asroma (to luigi_sslazio):

Hi Luigi, Roma won against Lazio last time, how do you feel?

--------------------------------------------------------------------------------
luigi_sslazio (to marcello_asroma):

Ciao Marcello! They got lucky, next time we'll show them! 😉 Forza Lazio!

--------------------------------------------------------------------------------
marcello_asroma (to luigi_sslazio):

Haha, keep dreaming, Luigi! 2-0 isn't luck. Forza Roma! 💛❤️ Gotta go, goodbye!

--------------------------------------------------------------------------------


In [11]:
from pprint import pprint
pprint(chat_result.summary)

("Marcello, a Roma fan, mentioned Roma's recent win against Lazio and teased "
 'Luigi, a Lazio fan. Luigi downplayed the loss, suggesting it was due to '
 "luck, and expressed confidence in Lazio's future success. Marcello replied "
 'with playful banter and ended the conversation.')
